# Module 1: RAG (Retrieval-Augmented Generation)

This module uses Amazon Bedrock Embedding model and TiDB Serverless Vector Search to retrieve the data. And then, use the LLM (Large Language Model) to generate the answer of the question.

Here, we demonstrate how easily you can build a RAG application using `pytidb` and TiDB Serverless.

> **Note:**
>
> - We already set the `SERVERLESS_CLUSTER_HOST`, `SERVERLESS_CLUSTER_PORT`, `SERVERLESS_CLUSTER_USERNAME`, `SERVERLESS_CLUSTER_PASSWORD`, and `SERVERLESS_CLUSTER_DATABASE_NAME` in the environment parameters.
> - We also granted the permission of using Amazon Bedrock for this lab. If you want to use this code snippet out of TiDB Labs platform, please set them beforehand.

## Install Dependencies

In [ ]:
%pip install -q \
    pytidb==0.0.10.dev1 \
    boto3==1.38.23 \
    litellm

## Initial the Clients of Database

In [ ]:
import os

from litellm import completion
from typing import Optional, Any
from pytidb import TiDBClient
from pytidb.schema import TableModel, Field
from pytidb.embeddings import EmbeddingFunction

db = TiDBClient.connect(
    host=os.getenv("SERVERLESS_CLUSTER_HOST"),
    port=int(os.getenv("SERVERLESS_CLUSTER_PORT")),
    username=os.getenv("SERVERLESS_CLUSTER_USERNAME"),
    password=os.getenv("SERVERLESS_CLUSTER_PASSWORD"),
    database=os.getenv("SERVERLESS_CLUSTER_DATABASE_NAME"),
    enable_ssl=True,
)

embedding_model = "bedrock/amazon.titan-embed-text-v2:0"

text_embedding_function = EmbeddingFunction(
    embedding_model,
    timeout=60
)

## Prepare the Context

In this case, contexts are the documents, use the openai embeddings model to get the embeddings of the documents, and store them in the TiDB.

In [ ]:
table_name = "documents"
class Document(TableModel, table=True):
    __tablename__ = table_name
    __table_args__ = {"extend_existing": True}
    id: int | None = Field(default=None, primary_key=True)
    text: str = Field(max_length=1024)
    embedding: Optional[Any] = text_embedding_function.VectorField(
        source_field="text",
    )

documents = [
    Document(id=1, text="TiDB is an open-source distributed SQL database that supports Hybrid Transactional and Analytical Processing (HTAP) workloads."),
    Document(id=2, text="TiFlash is the key component that makes TiDB essentially an Hybrid Transactional/Analytical Processing (HTAP) database. As a columnar storage extension of TiKV, TiFlash provides both good isolation level and strong consistency guarantee."),
    Document(id=3, text="TiKV is a distributed and transactional key-value database, which provides transactional APIs with ACID compliance. With the implementation of the Raft consensus algorithm and consensus state stored in RocksDB, TiKV guarantees data consistency between multiple replicas and high availability. "),
]

table = db.create_table(schema=Document, if_exists="overwrite")
table.bulk_insert(documents)

## Retrieve by Cosine Distance of Vectors

Get the relevant documents from the TiDB by comparing the embeddings of the question and the documents

In [ ]:
question = "what is TiKV?"

results = table.search(question).limit(1)
results.to_pandas()

## Generate the Answer

In [ ]:
from litellm import completion

llm_model = "bedrock/us.amazon.nova-lite-v1:0"

messages = [
    {"role": "system", "content": f"Please carefully answer the question by {str(results)}"},
    {"role": "user", "content": question}
]

llm_response = completion(
    model=llm_model,
    messages=messages,
)

print(llm_response.choices[0].message.content)